# Jupyter GraphQL Example

In this notebook, we will create a new Jupyter server and interact with it through the GraphQL endpoints.

## Start server

First, we will start up the new server and import a few helpers `q` and `s` to run GraphQL queries and subscriptions:

In [9]:
from jupyter_graphql.interactive import *

## Create kernel

Now we can make a new kernel and grab its id:

In [10]:
kernel_id = (await q("""
mutation {
    startKernel(input: {}) {
        kernel {
            id
        }
    }
}
"""))['startKernel']['kernel']['id']

Starting kernel: ['/usr/local/Caskroom/miniconda/base/envs/rtc/bin/python', '-m', 'ipykernel_launcher', '-f', '/private/var/folders/xn/05ktz3056kqd9n8frgd6236h0000gn/T/pytest-of-saul/pytest-268/test_stuff0/runtime/kernel-1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f.json']
Connecting to: tcp://127.0.0.1:61178
Connecting to: tcp://127.0.0.1:61175
Kernel started: 1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f
Kernel args: {'kernel_name': 'python3'}
activity on 1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f: status (starting)
Connecting to: tcp://127.0.0.1:61175
Connecting to: tcp://127.0.0.1:61174
activity on 1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f: status (busy)
activity on 1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f: status (idle)


## Get list of kernels

If we list our kernels, we will see it:

In [3]:
await q("""
query {
    kernels {
        id
        executionState
    }
}
""")

{'kernels': [{'id': 'kernel:9bcd086c-092e-460c-8766-86f0064d8bc0',
   'executionState': 'IDLE'}]}

## Get notified on execution state updates

We can set up a subscription to get notificed whenever the state of it updates:

In [4]:
s("""
subscription($id: ID!) {
    kernelExecutionStateUpdated(id: $id) {
        executionState
    }
}
""", id=kernel_id)

[{'kernelExecutionStateUpdated': {'executionState': 'RESTARTING'}},
 {'kernelExecutionStateUpdated': {'executionState': 'BUSY'}},
 {'kernelExecutionStateUpdated': {'executionState': 'IDLE'}},
 {'kernelExecutionStateUpdated': {'executionState': 'BUSY'}},
 {'kernelExecutionStateUpdated': {'executionState': 'IDLE'}}]

## Restart kernel

If we restart the kernel, we can see it updating:

In [5]:
await q("""
mutation($id: ID!) {
    restartKernel(input: {id: $id}) {
        kernel {
            executionState
        }
    }
}
""", id=kernel_id)

/usr/local/Caskroom/miniconda/base/envs/rtc/lib/python3.8/site-packages/jupyter_client/manager.py:357: FutureWarning: Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False).
  warnings.warn("Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False).",
Starting kernel: ['/usr/local/Caskroom/miniconda/base/envs/rtc/bin/python', '-m', 'ipykernel_launcher', '-f', '/private/var/folders/xn/05ktz3056kqd9n8frgd6236h0000gn/T/pytest-of-saul/pytest-268/test_stuff0/runtime/kernel-9bcd086c-092e-460c-8766-86f0064d8bc0.json']
Connecting to: tcp://127.0.0.1:61137
Kernel restarted: 9bcd086c-092e-460c-8766-86f0064d8bc0
activity on 9bcd086c-092e-460c-8766-86f0064d8bc0: status (busy)


{'restartKernel': {'kernel': {'executionState': 'RESTARTING'}}}

activity on 9bcd086c-092e-460c-8766-86f0064d8bc0: shutdown_reply
activity on 9bcd086c-092e-460c-8766-86f0064d8bc0: status (idle)
activity on 9bcd086c-092e-460c-8766-86f0064d8bc0: status (starting)
Connecting to: tcp://127.0.0.1:61134
Connecting to: tcp://127.0.0.1:61133
activity on 9bcd086c-092e-460c-8766-86f0064d8bc0: status (busy)
activity on 9bcd086c-092e-460c-8766-86f0064d8bc0: status (idle)


## Execute code

Now we can trying executing some code. We will use this example, that prints `2` and returns `"hello world"`

In [11]:
print(1 + 1)
"hello world"

2


'hello world'

In [12]:
execution = await q("""
mutation($kernel_id: ID!, $code: String!) {
    execute(input: {kernel: $kernel_id, code: $code}) {
        execution {
            id
            code
            kernel {
                id
            }
            status {
                __typename
                ... on ExecutionStateOK {
                    executionCount
                    data
                    metadata
                }
                ... on ExecutionStateError {
                    traceback
                    value
                    executionCount
                }
            }
            displays {
                __typename
                ... on DisplayStream {
                    name
                    text
                }
                ... on DisplayData {
                    data
                    metadata
                    displayID
                }
            }
            inputRequest {
                prompt
                password
            }
        }
    }
}
""",kernel_id=kernel_id, code="print(1 + 1)\n'hello world'")
execution

{'execute': {'execution': {'id': 'execution:3b62f546-0caa-11eb-81ab-faffc2200d70',
   'code': "print(1 + 1)\n'hello world'",
   'kernel': {'id': 'kernel:1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f'},
   'status': None,
   'displays': [],
   'inputRequest': None}}}

activity on 1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f: status (busy)
activity on 1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f: execute_input
activity on 1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f: stream
activity on 1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f: execute_result
Unhandled shell message: {'header': {'msg_id': '0beeea6e-2b591ba91befc857e84f59fe_8', 'msg_type': 'execute_reply', 'username': 'saul', 'session': '0beeea6e-2b591ba91befc857e84f59fe', 'date': datetime.datetime(2020, 10, 12, 16, 44, 44, 538464, tzinfo=tzutc()), 'version': '5.3'}, 'msg_id': '0beeea6e-2b591ba91befc857e84f59fe_8', 'msg_type': 'execute_reply', 'parent_header': {'msg_id': '25d63ab2-9692ef98479af5a85cc98415_1', 'msg_type': 'execute_request', 'username': 'saul', 'session': '25d63ab2-9692ef98479af5a85cc98415', 'date': datetime.datetime(2020, 10, 12, 16, 44, 44, 524761, tzinfo=tzutc()), 'version': '5.3'}, 'metadata': {'started': '2020-10-12T16:44:44.526333Z', 'dependencies_met': True, 'engine': 'f713d68e-66ce-42ab-ba38-bc5a153ffca5', 

Now we can query for the result of that execution:

In [13]:
await q("""
query($id: ID!) {
    execution(id: $id) {
            id
            code
            kernel {
                id
            }
            status {
                __typename
                ... on ExecutionStateOK {
                    executionCount
                    data
                    metadata
                }
                ... on ExecutionStateError {
                    traceback
                    value
                    executionCount
                }
            }
            displays {
                __typename
                ... on DisplayStream {
                    name
                    text
                }
                ... on DisplayData {
                    data
                    metadata
                    displayID
                }
            }
            inputRequest {
                prompt
                password
            }
        }
}
""",id=execution['execute']['execution']['id'])

{'execution': {'id': 'execution:3b62f546-0caa-11eb-81ab-faffc2200d70',
  'code': "print(1 + 1)\n'hello world'",
  'kernel': {'id': 'kernel:1fb1ceec-61fb-49bf-8e3b-9d9e61c69b1f'},
  'status': {'__typename': 'ExecutionStateOK',
   'executionCount': 1,
   'data': '{"text/plain": "\'hello world\'"}',
   'metadata': '{}'},
  'displays': [{'__typename': 'DisplayStream',
    'name': 'STDOUT',
    'text': '2\n'}],
  'inputRequest': None}}